In [1]:
from copy import deepcopy
from collections import Counter

In [2]:
import csv
with open('TriangleCornerCitations.csv', 'r') as csvfile:
    citations = list(csv.reader(csvfile, delimiter=','))[1:]
print(len(citations))

204


In [4]:
set([c[1] for c in citations])  # Identifiers for corner/triangle (extra: can also use "ascl: 1702.002")

{'10.21105/joss.00024',
 '10.5281/zenodo.10598',
 '10.5281/zenodo.11020',
 '10.5281/zenodo.45906',
 '10.5281/zenodo.53155',
 '2017ascl.soft02002F'}

In [5]:
all_ids = sum([[c[0], c[1]] for c in citations], [])

In [6]:
concept = '10.5281/zenodo.591491'  # Concept DOI
identities = [
    (('2017ascl.soft02002F', 'ads'), ('10.21105/joss.00024','doi')),
]
version = [
    # NOTE: Assuming the paper to be a "IsVersionOf" ConceptDOI
    (concept, '10.21105/joss.00024'),
    # New version release
    (concept, '10.5281/zenodo.53155'),
    (concept, '10.5281/zenodo.45906'),
    # Old two records on Zenodo (Triangle.py)
    (concept, '10.5281/zenodo.10598'),
    (concept, '10.5281/zenodo.11020'),
]

In [7]:
from utils import IDENTITY, VERSION, CITES, write_payloads

In [8]:
id_events = []
for src, tar in identities:
    rel = deepcopy(IDENTITY)
    rel['LinkPublicationDate'] = '2018-01-01'
    rel['Source']['Identifier']['ID'] = src[0]
    rel['Source']['Identifier']['IDScheme'] = src[1]
    rel['Target']['Identifier']['ID'] = tar[0]
    rel['Target']['Identifier']['IDScheme'] = tar[1]
    id_events.append(rel)

In [9]:
version_events = []
for src, tar in version:
    rel = deepcopy(VERSION)
    rel['LinkPublicationDate'] = '2018-01-01'
    rel['Source']['Identifier']['ID'] = src
    rel['Source']['Identifier']['IDScheme'] = 'doi'
    rel['Target']['Identifier']['ID'] = tar
    rel['Target']['Identifier']['IDScheme'] = 'doi'
    version_events.append(rel)

In [10]:
def expand_cit(c):
    def get_scheme(id_):
        return 'doi' if id_.startswith('10') else 'ads'
    src = c[0]
    src_scheme = get_scheme(src)
    tar = c[1]
    tar_scheme = get_scheme(tar)
    month, year = c[3].split('/')
    date = '{year}-{month}-01'.format(year=year, month=month)
    rel = deepcopy(CITES)
    rel['LinkPublicationDate'] = date
    rel['Source']['Identifier'] = {'ID': src, 'IDScheme': src_scheme}
    rel['Target']['Identifier'] = {'ID': tar, 'IDScheme': tar_scheme}
    rel['LinkProvider'] = [{'Name': 'SAO/NASA Astrophysics Data System'}]
    return rel    

In [12]:
citation_events = list(map(expand_cit, citations))

In [14]:
events = id_events + version_events + citation_events
write_payloads('events/cornerpy', events, chunk_size=100)